In [1]:
import hazm
import nltk
import pandas
from __future__ import unicode_literals
from hazm import *
import re 
from parsivar import Normalizer
from parsivar import Tokenizer
import pickle
import arabic_reshaper
import wget

ModuleNotFoundError: No module named 'hazm'

In [ ]:
df = pandas.read_excel('final_books.xlsx')
df.info()

: 

In [ ]:
df.iloc[502]['content']

In [ ]:
def remove_punctuations(text):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~،'''
    for char in text:
        if char in punctuations:
            text = text.replace(char, "")
    return text

In [ ]:
import hazm

normalizer = Normalizer()
lemmatizer = hazm.Lemmatizer()
stop_words = hazm.stopwords_list()

tokens = []
lemmatized_tokens = []
for index, row in df.iterrows():
    content = row['content'] 
    norm_content = normalizer.normalize(str(content))
    norm_content_no_punctuations = remove_punctuations(norm_content)
    tokenized_sentences = hazm.sent_tokenize(norm_content_no_punctuations)
    content_tokenized_words = []
    for sentence in tokenized_sentences:
        try:
            tokenized_words = hazm.word_tokenize(sentence)
            filtered_words = [word for word in tokenized_words if word not in stop_words]
            content_tokenized_words.extend(filtered_words)
            for word in filtered_words:
                lemmatizedWord = lemmatizer.lemmatize(word)
                lemmatized_tokens.append((index, lemmatizedWord))
        except:
            print("error")  
    tokens.extend(content_tokenized_words)

In [ ]:
def create_word_index(arr):
    word_index = {}
    for doc_id, word in arr:
        if word in word_index:
            word_index[word].append(doc_id)
        else:
            word_index[word] = [doc_id]
    return dict(sorted(word_index.items(), key=lambda x: len(x[1]), reverse=True))

In [ ]:
word_index = create_word_index(lemmatized_tokens)

In [ ]:
first_key, first_value = next(iter(word_index.items()))
print(first_key, first_value[0:100])

In [ ]:
def sort_dict_values(d):
    sorted_dict = {}
    for k in d:
        sorted_dict[k] = sorted(set(d[k]), key=d[k].count, reverse=True)
    return sorted_dict

In [ ]:
sorted_word_index = sort_dict_values(word_index)
first_key, first_value = next(iter(sorted_word_index.items()))
print(first_key, first_value[0:10])

In [ ]:
# Save the word_dictionaries object to a file
with open('index_dict.pkl', 'wb') as f:
    pickle.dump(sorted_word_index, f)